# Final Project: Admission Prediction from NHAMCS
## Initial LR model
### DS5559: Big Data Analysis
### Thomas Hartka, Alicia Doan, Michael Langmayr
Created: 6/28/2020 
  
In this notebook creates and analyzes logistic regression model for predicting hospital admission in the NHAMCS database.  For this iniital model, categorical variables will be represented with one-hot encoding and the reason for visit (RFV) variables will be ignored.  The RFV varaibles will be ignored because there are hundreds of different potential values and we have not yet developed a way to categorize them yet.  Binary variables were previously converted to 0/1.  

## Configuration

In [3]:
# set data directory
data_dir = "../data"
results_dir = "../results"

In [4]:
# import python libraries
import os
import pandas as pd
import numpy as np
from functools import reduce

In [5]:
# set up pyspark
from pyspark.sql import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.getOrCreate()

## Read in data and create data sets

In [6]:
NHAMCS = spark.read.parquet(data_dir + "/NHAMCS_processed.2007-2017")

In [8]:
# first remove RFV variables
NHAMCS = NHAMCS.drop('RFV1','RFV2','RFV3','RFV4','RFV5')

In [9]:
# split into training and testing set
training, testing = NHAMCS.randomSplit([0.8, 0.2], 42)

In [12]:
NHAMCS.select(NHAMCS.columns[10:20]).show(5)

+------+-----+---------+-----+------+----+------+-----+---+----+
|BPDIAS|POPCT|PAINSCALE|ALZHD|ASTHMA| CAD|CANCER|CEBVD|CHF| CKD|
+------+-----+---------+-----+------+----+------+-----+---+----+
|    93|   99|        8| null|  null|null|  null|    0|  0|null|
|    75|   99|        6| null|  null|null|  null|    0|  0|null|
|    82|   98|        2| null|  null|null|  null|    0|  0|null|
|    76|   98|        0| null|  null|null|  null|    0|  0|null|
|    69|   97|        0| null|  null|null|  null|    0|  0|null|
+------+-----+---------+-----+------+----+------+-----+---+----+
only showing top 5 rows



## Set up pipeline

In [ ]:
# process rating data
ohe = OneHotEncoder(inputCol="rating", outputCol="rc")